# Overdensity counter using Gadget interface

In [1]:
import os
from os import path
from os import system
import snaptools
from snaptools import plot_tools
import numpy as np
import subprocess
import random
import time
from snaptools import snapshot
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from IPython.display import display, Math
from scipy.optimize import fsolve
import matplotlib.ticker as mticker
import shutil


Failed to import CuPy.

If you installed CuPy via wheels (cupy-cudaXXX or cupy-rocm-X-X), make sure that the package matches with the version of CUDA or ROCm installed.

On Linux, you may need to set LD_LIBRARY_PATH environment variable depending on how you installed CUDA/ROCm.
On Windows, try setting CUDA_PATH environment variable.

Check the Installation Guide for details:
  https://docs.cupy.dev/en/latest/install.html

CUDA Path: c:\Users\thorb\miniconda3\envs\uni
DLL dependencies:
  KERNEL32.dll -> C:\WINDOWS\System32\KERNEL32.DLL
  MSVCP140.dll -> c:\Users\thorb\miniconda3\envs\uni\MSVCP140.dll
  VCRUNTIME140.dll -> c:\Users\thorb\miniconda3\envs\uni\VCRUNTIME140.dll
  VCRUNTIME140_1.dll -> c:\Users\thorb\miniconda3\envs\uni\VCRUNTIME140_1.dll
  api-ms-win-crt-convert-l1-1-0.dll -> C:\WINDOWS\System32\ucrtbase.dll
  api-ms-win-crt-environment-l1-1-0.dll -> C:\WINDOWS\System32\ucrtbase.dll
  api-ms-win-crt-filesystem-l1-1-0.dll -> C:\WINDOWS\System32\ucrtbase.dll
  api-ms-win-crt-

c:\users\thorb\downloads\cupy\cupy\_environment.py:445: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy, cupy-cuda111

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''


In [2]:
print(timeNow := str(time.time()).split('.')[0])

1680075072


In [3]:
timeIndex = 0

boxsize = 1 #Mpc/h
runCnts = 3 #number of runs to coserve memory usage
gridsize = 128
ntypes = 6
timeAtPS = 0.00990099 #rs=100
#timeAtPS = 0.09090898 #rs=10
rs = 100
unitLength_in_cm = '3.08568e+24'
hubble = 100


In [4]:
sshAddress = '---@astro01.hpc.ku.dk'
sshPassword = ''
PSfile = 'cmb_code_wmap7_spectrum.txt'
gadgetPath = 'gadget4/examples/createSnap'
gadgetSuperPath_HPC = 'gadget4'
gadgetSubPath_HPC = 'examples/createSnap'
WSLHome = r'\\wsl.localhost\\Ubuntu\\home\\thorbjorn'
snapDestination_WSL = '/mnt/d/Dokumenter/Skole/1.års/camb-pipeline/2_snapshots'
snapDestination_local = '2_snapshots'
HPCEnvironmentCommands = [
    "umask 027",
    "module load astro",
    "module load python/anaconda3",
    "module load intel intelmpi",
    "module load gcc",
    "module load git"
]

In [5]:
def command(cmd):
    p = subprocess.Popen(cmd, text=True, stdout=subprocess.PIPE)
    while p.poll() is None:
        print(p.stdout.readline())
    return p.stdout.read()
def returnCommand(cmd):
    return subprocess.Popen(cmd, text=True, stdout=subprocess.PIPE).stdout.read()
def sshCommand(cmd):
    newCommand = f'wsl --cd ~ sshpass -p {sshPassword} ssh {sshAddress} "{cmd}"'
    return command(newCommand)
def sshReturnCommand(cmd):
    newCommand = f'wsl --cd ~ sshpass -p {sshPassword} ssh {sshAddress} "{cmd}"'
    return returnCommand(newCommand)
def sshDownloadFile(sshPath, localPath):
    command(f"wsl --cd ~ sshpass -p '{sshPassword}' scp -rp {sshAddress}:{sshPath} {localPath}")

def sshUploadFile(localPath, sshPath):
    command(f"wsl --cd ~ sshpass -p '{sshPassword}' scp -rp {localPath} {sshAddress}:{sshPath}")

def editParamFile(param2edit, value):
    configDir = os.path.join(WSLHome, 'param.txt')
    with open(configDir) as f:
        lines = f.readlines()
        paramFound = False
        for i in range(len(lines)):
            if lines[i].startswith(param2edit):
                lines[i] = f'{param2edit}             ' + str(value) + r"   %editted by Python <3"  + '\n'
                paramFound = True
                break
        if not paramFound:
            lines.append(f'{param2edit}             ' + str(value) + r"   %editted by Python <3"  + '\n')
    with open(configDir, 'w') as f:
        f.writelines(lines)
def rmParamInParamFile(param2rm):
    configDir = os.path.join(WSLHome, 'param.txt')
    with open(configDir) as f:
        lines = f.readlines()
        paramFound = False
        for i in range(len(lines)):
            if lines[i].startswith(param2rm):
                lines[i] = ''
                paramFound = True
                break
    with open(configDir, 'w') as f:
        f.writelines(lines)

def editConfigFile(param2edit, value):
    configDir = os.path.join(WSLHome, 'Config.sh')
    with open(configDir) as f:
        lines = f.readlines()
        for i in range(len(lines)):
            if lines[i].startswith("    " + param2edit):
                lines[i] = "    " + f'{param2edit}=' + str(value) + "   #editted by Python <3" + '\n'
    with open(configDir, 'w') as f:
        f.writelines(lines)

In [6]:
#take data from local dir
RSs = np.loadtxt('RSs.txt')
times = np.loadtxt('times.txt')
rs = RSs[timeIndex]
timeAtPS = times[timeIndex]
specs = os.listdir('1. spectrogram')
specs = [e for e in specs if str(rs) in e]
print(specs)
if len(specs) != 1:
    raise Exception('No or multiple spectrograms found for rs=' + str(rs))
spec = np.loadtxt('1. spectrogram/' + specs[0])
PSFile = f'spec_i{timeIndex}.txt'
#copy PS to local
shutil.copyfile('1. spectrogram/' + specs[0], WSLHome + '\\' + 'spec.txt')
sshUploadFile('~/spec.txt', f'~/{gadgetPath}/{PSFile}')
#os.remove(specs[0])


['camb-pipeline-spec-z10.000013420016371.txt']




















































































In [7]:
#update config and param files
sshDownloadFile(f'~/{gadgetPath}/param.txt', 'param.txt')
sshDownloadFile(f'~/{gadgetPath}/Config.sh', 'Config.sh')
sshDownloadFile(f'~/{gadgetPath}/outputs.txt', 'outputs.txt')
editParamFile('BoxSize', boxsize)
editParamFile('GridSize', gridsize)
editParamFile('NSample', gridsize)
editParamFile('PowerSpectrumFile', PSfile)
editParamFile('TimeBegin', timeAtPS)
editParamFile('Hubble ', hubble)
editParamFile('UnitLength_in_cm', unitLength_in_cm)
editConfigFile('PMGRID', gridsize)
editConfigFile('NGENIC', gridsize)
editConfigFile('NTYPES', ntypes)
editConfigFile('STOP_AFTER_STEP', 1)
for i in range(ntypes):
    editParamFile(f'SofteningClassOfPartType{i}', 0)
for i in range(ntypes+1, 11):
    rmParamInParamFile(f'SofteningClassOfPartType{i}')

with open(os.path.join(WSLHome, 'outputs.txt'), 'w') as f:
    f.writelines([f'{timeAtPS}'])
sshUploadFile('param.txt', f'~/{gadgetPath}/param.txt')
sshUploadFile('Config.sh', f'~/{gadgetPath}/Config.sh')
sshUploadFile('outputs.txt', f'~/{gadgetPath}/outputs.txt')

In [8]:
#build gadget
sshCommand(f'cd ~/{gadgetSuperPath_HPC}; make -j 20 DIR={gadgetSubPath_HPC}/')

Build configuration:

SYSTYPE: "Generic-gcc"

CONFIG: examples/createSnap//Config.sh

EXEC: examples/createSnap//Gadget4



python buildsystem/config.py examples/createSnap//Config.sh examples/createSnap//build /lustre/hpc/astro/bcm464/gadget4 src

mpicxx  -std=c++11  -ggdb -O3 -march=native  -Wall -Wno-format-security    -I/hdf5/build/include -I/gsl/build/include -I/fftw3/build/include    -Iexamples/createSnap//build -Isrc -c src/main/begrun.cc -o examples/createSnap//build/main/begrun.o

mpicxx  -std=c++11  -ggdb -O3 -march=native  -Wall -Wno-format-security    -I/hdf5/build/include -I/gsl/build/include -I/fftw3/build/include    -Iexamples/createSnap//build -Isrc -c src/main/init.cc -o examples/createSnap//build/main/init.o

mpicxx  -std=c++11  -ggdb -O3 -march=native  -Wall -Wno-format-security    -I/hdf5/build/include -I/gsl/build/include -I/fftw3/build/include    -Iexamples/createSnap//build -Isrc -c src/main/main.cc -o examples/createSnap//build/main/main.o

mpicxx  -std=c++11  -

''

In [9]:
def sshLs(path):
    return "".join(list(sshReturnCommand(f'cd ~/{path}; ls'))).split("\n")[:-1]

seeds = random.sample(range(100001,199999), runCnts)
snapDirs = []
print(runCnts)
for i in range(runCnts):
    #set seed
    
    sshDownloadFile(f'~/{gadgetPath}/param.txt', 'param.txt')
    editParamFile('Seed', seeds[i])
    sshUploadFile('param.txt', f'~/{gadgetPath}/param.txt')
    sshCommand(f'cd ~/{gadgetPath}; rm ewald_table*')
    #run gadget
    path = gadgetSuperPath_HPC + "/" + gadgetSubPath_HPC
    #listdir
    dirList = sshLs(path)
    print(dirList)
    oldDirList = [d for d in dirList if "slurm-" in str(d)]
    print("oldDirList", oldDirList)
    sshCommand(f'cd ~/{path}; sbatch job.sh')
    #wait for job to start
    print("Waiting for job to start", end='')
    timer = 0
    while True:
        dirList = sshLs(path)
        newDirList = [d for d in dirList if "slurm-" in d]
        if len(newDirList) > len(oldDirList):
            break
        time.sleep(4)
        print('.', end='')
        timer += 4
        print(timer, end='')
        if timer > 100000:
            raise TimeoutError("Job did not start")
    print()
    timer = 0
    #wait for job to finish
    print("Waiting for job to finish", end='')
    newSlurm = [d for d in newDirList if d not in oldDirList][0]
    running = True
    while running:#check if one of the lines in the slurm file contains "bye!"
        slurmFile = "".join(sshReturnCommand(f'cd ~/{path}; cat {newSlurm}')).split("\n")
        print(slurmFile)
        for l in slurmFile[::-1][:10]:
            if "bye!" in l:
                running = False
        time.sleep(1)
        print('.', end='')
        timer += 1
        if timer > 1000:
            raise TimeoutError("Job did not finish")
    print()
    print("Job finished")
    #copy and remove snapdir_000
    sshDownloadFile(f'~/{path}/output/snapdir_000/', f'snapdir_{timeNow}/')
    sshCommand(f'rm -r ~/{path}/output/snapdir_000/')
    dirName = f'snapdir_s{seeds[i]}_t{timeNow}'
    command(f'wsl --cd ~ mv snapdir_{timeNow} {snapDestination_WSL}/{dirName}')
    snapDirs.append(dirName)

    

3
































































































































































































































































































































































['Config.sh', 'Config.sh.save', 'Gadget4', 'PlotPowerSpec.pro', 'build', 'cambspec.py', 'cmb_code_wmap7_spectrum.txt', 'job.sh', 'matter_power_spectrum.png', 'output', 'outputs.txt', 'param.txt', 'param.txt-usedvalues', 'param.txt.save', 'slurm-38543050.out', 'slurm-38543356.out', 'slurm-38543533.out', 'slurm-38543560.out', 'slurm-38543611.out', 'slurm-38543747.out', 'slurm-38543750.out', 'slurm-38543765.out', 'slurm-38543773.out', 'slurm-38543800.out', 'slurm-38543805.out', 'slurm-38543808.out', 'slurm-38543812.out', 'slurm-38543910.out', 'slurm-38543914.out', 'slurm-38543998.out', 'slurm-38544001.out', 'slurm-38544166.out', 'slurm-38544

In [10]:
#boxsize /= 1000

In [11]:
do3d = True
p = snapDestination_local
if p == '':
    p = os.getcwd()
snapDirs = os.listdir(p)
snapDirs = [d for d in snapDirs if "snapdir_s" in d]
snapDirs = [d for d in snapDirs if ".txt" not in d]
times = [int(d.split('_t')[-1]) for d in snapDirs]
seeds = [int(d.split('_s')[-1].split('_t')[0]) for d in snapDirs]
latestTime = max(times)
snapDirs = [d for d in snapDirs if str(latestTime) in d]
binSpace = np.arange(2,gridsize,1)
sigmas = []
avgs = []
keptBins = []
for snapDir in snapDirs:
    snapParts = os.listdir(os.path.join(snapDestination_local, snapDir))
    snapParts = [os.path.join(snapDestination_local, snapDir, p) for p in snapParts]
    snapParts = [p for p in snapParts if ".hdf5" in p]
    #print(snapParts)
    snap = snapshot.Snapshot(snapParts[0])
    print(snap.header)
    #print(snap)
    #print(snap.pos['halo'])
    sigmas.append(np.zeros(len(binSpace)))
    keptBins.append([[] for i in range(len(binSpace))])
    if do3d:
        for i,g in enumerate(binSpace):
            print("grid: ", g)
            binned, _ = plot_tools.plot_combined_3D(snapParts, parttype='halo',xlen=(0,boxsize),ylen=(0,boxsize),zlen=(0,boxsize),colormap='plasma',NBINS=g,gadgetGridsize=gridsize,len2kpc=1000,visualize=False)
            keptBins[-1][i] = binned
            #calc avg
            if i == 0:
                avgs.append(np.sum(binned['Z2'])/(boxsize**3))
            #calculate sigma
            #print(binned)
            sigma = np.std(binned['Z2'])
            sigmas[-1][i] = sigma
    else:
        for i,g in enumerate(binSpace):
            print("grid: ", g)
            binned, _ = plot_tools.plot_combined(snapParts, parttype='halo',xlen=(0,boxsize),ylen=(0,boxsize),colormap='plasma',NBINS=g,gadgetGridsize=gridsize,len2kpc=1000,visualize=False)
            keptBins[-1][i] = binned
            #rename plot
            oldPlotName = os.path.join('plot', f'{snapParts[0]}.png')
            newPlotName = os.path.join('plot', f'{snapParts[0]}_g{g}.png')
            #if newPlotName exists, remove it
            if os.path.exists(newPlotName):
                os.remove(newPlotName)
            os.rename(oldPlotName, newPlotName)
            #calc avg
            if i == 0:
                avgs.append(np.sum(binned['Z2'])/(boxsize**3))
            #calculate sigma
            sigma = np.std(binned['Z2'])
            sigmas[-1][i] = sigma


    

{'boxsize': 1.0, 'Git_commit': b'6012ae8eee45969e190ad59e842ff4f9ce9bf416', 'Git_date': b'Mon Mar 6 17:31:49 2023 +0100', 'massarr': array([0.00000000e+00, 4.07516425e-06, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00]), 'filenum': 30, 'npart': array([    0, 87380,     0,     0,     0,     0], dtype=uint64), 'nall': array([      0, 2097152,       0,       0,       0,       0], dtype=uint64), 'redshift': 10.000013420016371, 'time': 0.09090898000000001}
grid:  2
grid:  3
grid:  4
grid:  5
grid:  6
grid:  7
grid:  8
grid:  9
grid:  10
grid:  11
grid:  12
grid:  13
grid:  14
grid:  15
grid:  16
grid:  17
grid:  18
grid:  19
grid:  20
grid:  21
grid:  22
grid:  23
grid:  24
grid:  25
grid:  26
grid:  27
grid:  28
grid:  29
grid:  30
grid:  31
grid:  32
grid:  33
grid:  34
grid:  35
grid:  36
grid:  37
grid:  38
grid:  39
grid:  40
grid:  41
grid:  42
grid:  43
grid:  44
grid:  45
grid:  46
grid:  47
grid:  48
grid:  49
grid:  50
grid:  51
grid:  52
grid:  53
grid:  5

In [18]:
dataset = {
    'virializationRate' : [],
    'counts' : [],
    'stds' : []
}
from math import ceil
import scipy.integrate as integrate
plt.style.use('ggplot')

colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan', 'tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan', 'tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan', 'tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan', 'tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan', 'tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan', 'tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan', 'tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan', 'tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan', 'tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan', 'tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan', 'tab:blue', 'tab:orange',]

fig = plt.figure(figsize=(15,10))
ax = fig.add_subplot(111)
pltCounter = 0
gsToPlot = [gridsize-1]
while 1/gsToPlot[-1]*boxsize <= 1/4*boxsize:
    gsToPlot.append(gsToPlot[-1]//2)
for i,g in enumerate(binSpace):
    if g not in gsToPlot:
        continue
    else:
        pltCounter += 1
    data = keptBins[0][i]['Z2'].copy()
    mean = np.mean(data)
    virRate = data*(1.0482)/(mean*18*(np.pi**2))
    flatVirRate = virRate.flatten()
    
    bins = np.linspace(min(flatVirRate), max(flatVirRate), ceil(np.sqrt(len(flatVirRate))))
    
    count, bins = np.histogram(flatVirRate, bins=bins)

    
    toSum = [count]
    
    for k in range(1, len(snapDirs)):
        data = keptBins[k][i]['Z2'].copy()
        mean = np.mean(data)
        virRate = data*(1.0482)/(mean*18*(np.pi**2))
        flatVirRate = virRate.flatten()
        count, bins = np.histogram(flatVirRate, bins=bins)
        toSum.append(count)
    
    stds = np.std(toSum, axis=0)
    means = np.mean(toSum, axis=0)

    binRange = bins[-1] - bins[0]
    means /= (binRange/len(bins))
    
    binWidthkpc = boxsize*1000/g
    
    bins_lastRemoved = np.array([(bins[i]+bins[i+1])/2 for i in range(len(bins)-1)])
    #rebinning
    nonZeroInds = np.where(means > 0)
    means = means[nonZeroInds]
    stds = stds[nonZeroInds]
    bins_lastRemoved = bins_lastRemoved[nonZeroInds]
    bins_rebinned = bins_lastRemoved
    means_rebinned = means
    stds_rebinned = stds
    #print("rebinning...")
    while len(bins_rebinned) > 20:#!!bad rebinning
        """print("binmax= " + str(bins_rebinned[-1]))
        print("bins= " + str(len(bins_rebinned)))"""
        bins_rebinned = np.array([np.mean(bins_rebinned[j:j+2]) for j in range(0,len(bins_rebinned)-1,2)])
        means_rebinned = np.array([np.mean(means_rebinned[j:j+2]) for j in range(0,len(means_rebinned)-1,2)])
        stds_rebinned = np.array([np.sqrt(stds_rebinned[j]**2+stds_rebinned[j+1]**2) for j in range(0,len(stds_rebinned)-1,2)])
    #print("rebinning done")
    
    print("zeropointCount= " + str(len(means[means <= 0])))
    plt.errorbar(bins_rebinned, means_rebinned, yerr=stds_rebinned, capsize=5, color=colors[pltCounter],fmt='none')
    plt.plot(bins_rebinned, means_rebinned, label=f'r={round(binWidthkpc,1)}', marker='.', linewidth=0, markersize=10,color=colors[pltCounter])
    toSave = np.array([bins_rebinned, means_rebinned, stds_rebinned]).T
    np.savetxt(f'3_virilizationplots/data/bs{boxsize}_bw{binWidthkpc}_rs{rs}_gs{g}_data.txt', toSave, header='bin mean std')
    #make linear regime aproximation
    if len(means_rebinned) > 0:
        peakInd = np.argmax(means_rebinned)
        threeQuarters = (bins_rebinned[peakInd] + bins_rebinned[-1])/2
        linearRegime_bins = bins_rebinned[bins_rebinned >= threeQuarters]
        if len(linearRegime_bins) >= 3:
            linearRegime_means = means_rebinned[bins_rebinned >= threeQuarters]
            linearRegime_stds = stds_rebinned[bins_rebinned >= threeQuarters]
            #fit linear regime
            #linear = lambda x, a, b: a*x + b
            linear_notlog = lambda x, a, b: np.power(10,(a*x+b))
            #exp = lambda x, a, b, c: np.exp(a*x)*b+c
            #logOfMeans = np.log10(linearRegime_means)
            #validsIndeces = np.where(np.isfinite(logOfMeans))
            #logOfMeans = logOfMeans[validsIndeces]
            #binsForFit = linearRegime_bins[validsIndeces]
            #logOfStds = np.log10(linearRegime_stds[validsIndeces])
            #popt, pcov = curve_fit(linear, binsForFit, logOfMeans, sigma=logOfStds)
            popt, pcov = curve_fit(linear_notlog, linearRegime_bins, linearRegime_means, sigma=linearRegime_stds)
            print(f'linear fit, a={round(popt[0], 3)}'
                    f' with error {round(np.sqrt(pcov[0][0]), 3)}')
            print(f'linear fit, b={round(popt[1], 3)}'
                    f' with error {round(np.sqrt(pcov[1][1]), 3)}')
            """print(f'linear fit, c={round(popt[2], 3)}'
                    f' with error {round(np.sqrt(pcov[2][2]), 3)}')"""
            a_ = popt[0]
            b_ = popt[1]
            #c_ = popt[2]
            a_sigma = np.sqrt(pcov[0][0])
            b_sigma = np.sqrt(pcov[1][1])
            #c_sigma = np.sqrt(pcov[2][2])
            toSave = np.array([a_, b_, a_sigma, b_sigma]).T
            np.savetxt(f'3_virilizationplots/data/bs{boxsize}_bw{binWidthkpc}_rs{rs}_gs{g}_fit.txt', toSave, header='a b a_sigma b_sigma')
            if a_ < 0:
                linear_notlog = lambda x, a, b: np.power(10,(a*x+b))
                #exp_notlog = lambda x, a, b, c: np.power(10,(b*np.exp(x*a)+c))
                plt.plot(linearRegime_bins, linear_notlog(linearRegime_bins, *popt), label=f'linear fit, slope={round(popt[0], 3)}', linewidth=2, color=colors[pltCounter])
                #integrate exp for 1 to infinity
                
                expWithVars = lambda x: linear_notlog(x, a_, b_)
                linearValue = integrate.quad(expWithVars, 1, np.inf)[0]
                sigmalinearValue = np.sqrt((10**b_*10**a_*(1/a_))**2 *b_sigma**2 + (10**b_*10**a_*(1/a_)-1/(a_**2)*10**b_*10**a_*(1/np.log(10)))**2 *a_sigma**2)
                print(r'virilization prob. $1Mpc^3$' + '%.1E ± %.1E' % (linearValue, sigmalinearValue))
                plt.plot([],[],marker='.', linewidth=0, markersize=10, color=colors[pltCounter], label=r'Vir. prob. @$1Mpc^3$:   ' + '%.1E ± %.1E' % (linearValue, sigmalinearValue))
    
    

ax.set_yscale('log')
plt.legend()
plt.xlabel("virialization rate")
plt.ylabel("count")
plt.xlim(0, np.max(bins_rebinned)+0.01)
plt.title("virialization rate distribution")
plt.savefig(f"3_virilizationplots\\{int(boxsize*1000)}kpc\\virRate_rs{rs}_g{gridsize}_b{boxsize*1000}.png")
plt.show()
plt.close()
plt.clf()
        
        

zeropointCount= 0
zeropointCount= 0
linear fit, a=1.0 with error inf
linear fit, b=1.0 with error inf
zeropointCount= 0
linear fit, a=-52.003 with error 4.775
linear fit, b=5.257 with error 0.152
virilization prob. $1Mpc^3$1.5E-49 ± 1.7E-48
zeropointCount= 0
linear fit, a=-34.273 with error 2.706
linear fit, b=5.62 with error 0.153
virilization prob. $1Mpc^3$2.8E-31 ± 1.8E-30
zeropointCount= 0
linear fit, a=-30.113 with error 1.629
linear fit, b=6.783 with error 0.13
virilization prob. $1Mpc^3$6.8E-26 ± 2.6E-25


c:\Users\thorb\miniconda3\envs\uni\lib\site-packages\scipy\optimize\_minpack_py.py:803: RuntimeWarning: divide by zero encountered in true_divide
  transform = 1.0 / sigma
c:\Users\thorb\miniconda3\envs\uni\lib\site-packages\scipy\optimize\_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


zeropointCount= 0
linear fit, a=-26.784 with error 0.269
linear fit, b=8.925 with error 0.03
virilization prob. $1Mpc^3$2.2E-20 ± 1.4E-20


C:\Users\thorb\AppData\Local\Temp\ipykernel_11764\1826289545.py:128: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [19]:
plt.style.use('ggplot')
#save sigmas and averages and binspace
#print(r"$\hat{\rho}$", np.mean(avgs))
display(Math(r"$\hat{\rho}$ = " + str(np.mean(avgs)/((3.262e+3)**6)) + " $M_\odot$ / $ly^3$"))
binSize = 1/binSpace*boxsize*3.262e+6
"""print(runCnts)
runCnts = 4"""
sigmasMod = sigmas.copy()[:runCnts]
print(np.shape(sigmasMod))
for i in range(runCnts):
    #divide by binspace**3
    #sigmasMod[i] = sigmas[i] * binSpace**3
    sigmasMod[i] = sigmasMod[i] / (binSize**3)
    #sigmasMod[i] = sigmasMod[i] / avgs[i]
sigmasAvg = np.mean(sigmasMod, axis=0)
sigmasStd = np.std(sigmasMod, axis=0)/np.sqrt(runCnts)
fig = plt.figure()
ax = plt.axes()
#plt.errorbar(binSpace, sigmasAvg, yerr=sigmasStd, marker='.', label='sigma',markersize=1, capsize=1.5, capthick=1, fmt='none', elinewidth=1, ecolor='black')
#plt.loglog(binSize, sigmasAvg, marker='.', label='sigma',markersize=5,linewidth=0,color='tab:orange')
plt.errorbar((binSize), sigmasAvg, yerr=sigmasStd, marker='.', label='std',markersize=1, capsize=1.5, capthick=1, fmt='none', elinewidth=1, ecolor='black')
ax.set_xscale('log')
ax.set_yscale('log')
linear = lambda x, a, b: a*x + b
logOfMeans = np.log10(sigmasAvg)
logOfMods = np.log10(binSize)
validsIndeces = np.where(np.isfinite(logOfMeans))
logOfMeans = logOfMeans[validsIndeces]
logOfMods = logOfMods[validsIndeces]
logOfStds = np.log10(sigmasStd[validsIndeces])
popt, pcov = curve_fit(linear, logOfMods, logOfMeans, sigma=logOfStds)
linearAntiLogLog = lambda x, a, b: np.power(10,b)*np.power(x,a)
plt.plot(binSize, linearAntiLogLog(binSize, *popt), label='linear fit, slope={}'.format(round(popt[0], 3)), linewidth=2)
print("linear fit slope: " + str(popt[0]))
print("linear fit intercept: " + str(popt[1]))

plt.legend()
plt.title(r'$\sigma$ overdensities, $z={}$'.format(rs))
plt.xlabel('log10(gridsize) [log10(ly)]]')
#plt.ylabel(r'$\sigma$ [$M_{\odot}/h$]')
plt.ylabel(r'log10($\sigma_{\rho}$) [log10($M_{\odot}/ly^3$)]')
plt.savefig(f'3_virilizationplots\\{int(boxsize*1000)}kpc\\sigma_rs{rs}_bs{boxsize*1000}.png')
plt.show()
plt.close()
plt.clf()
    

<IPython.core.display.Math object>

(3, 126)
linear fit slope: -0.31652627641693987
linear fit intercept: -6.961559467697004


C:\Users\thorb\AppData\Local\Temp\ipykernel_11764\1149639039.py:43: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
